In [ ]:
from detectdd.auth_bigquery import BigQueryClient
%load_ext autoreload
%autoreload 2

import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from detectdd.config import *

In [ ]:
# def get_client(c,p):
#     return bigquery.Client(p, c)
# 
# def get_gcreds(scopes = None):
#     if scopes is None:
#         scopes = ["https://www.googleapis.com/auth/bigquery"]
#     return google.auth.default(
#         scopes )
# 
# def get_authed_client():
#     cred = get_gcreds()
#     print(*cred)
#     return get_client(*cred)

def main():
    mimic_drugs = f"""
        SELECT itemid, label, abbreviation, category, unitname
        FROM {icu_d_items}
        WHERE linksto='inputevents' """

    mimic_job = BigQueryClient.auth().query(mimic_drugs)
    df = mimic_job.to_dataframe()
    df.label = df.label.fillna("")
    df.label = df.label.apply(str.lower)
    return df

icu_drugs = main()

In [ ]:
def read_drugs():
    df = pd.read_csv(data_dir / "NDC_product_table.csv", encoding='ISO-8859-1')
    df.NONPROPRIETARYNAME = df.NONPROPRIETARYNAME.fillna("")
    df.NONPROPRIETARYNAME = df.NONPROPRIETARYNAME.apply(str.lower)
    df.PROPRIETARYNAME = df.PROPRIETARYNAME.fillna("")
    df.PROPRIETARYNAME = df.PROPRIETARYNAME.apply(str.lower)
    return df
ndc_drug_synonyms = read_drugs()
ndc_drug_synonyms

In [ ]:
from detectdd.drug_index import DrugIndex

drug_index = DrugIndex.init_with_drugbank()


In [ ]:
def read_hosp_drugs():
    sql_hosp_drugs_query = """
    SELECT """
    BigQueryClient.auth()

In [ ]:
def match():
    df = icu_drugs.merge(
        ndc_drug_synonyms.drop_duplicates(subset="NONPROPRIETARYNAME"), left_on= ['label'], right_on=['NONPROPRIETARYNAME'], how='left', indicator=True)[['itemid', 'label', 'category', 'PRODUCTID', 'NONPROPRIETARYNAME']]\
        .merge(ndc_drug_synonyms.drop_duplicates(subset="PROPRIETARYNAME"), left_on=['label'], right_on='PROPRIETARYNAME', how='left', indicator=True)
    print(df.count())
    return df


matched = match()
matched

In [ ]:

def fuzzy_match(str1, str2):
    return fuzz.token_set_ratio(str1, str2)

def match_dataframe(df1, key1, df2, key2, threshold=90):
    matches = []
    # matches = pd.DataFrame(columns=['index', 'label', 'norm_label', 'match0', 'match1'])
    for i, row in df1.iterrows():
        drug_name = row[key1]
        fast = True
        to_match = df2[key2]
        if fast:
            to_match = to_match.loc[to_match.str.startswith(drug_name[0])] # speed up fuzzy matching by only considering synonyms that start with the same letter

        match = process.extractOne(drug_name, to_match, scorer=fuzzy_match)

        if match[1] >= threshold:
            print(f"Found match with score ({str(match[1])}) : {row[key1]} - {match[0]}")
            matches.append([i, row['itemid'],row[key1], match[0], match[1], match[2]])
        print(i)

    return pd.DataFrame(matches, columns=['index', 'itemid', 'label', 'norm_label', 'score', 'norm_index'])



def fuzzy_merge():
    medications = icu_drugs.loc[(~matched['category'].isin(['Medications']))]
    return match_dataframe(medications, "label", ndc_drug_synonyms, "PROPRIETARYNAME")

fuzzy_matched = fuzzy_merge()
fuzzy_matched

In [ ]:
fuzzy_matched

In [ ]:
m1 = ndc_drug_synonyms['PROPRIETARYNAME'].str.startswith("glyc")
ndc_drug_synonyms.loc[m1]


In [ ]:
m1 = ~matched['PRODUCTID_x'].isnull()
m2 = ~matched['PRODUCTID_y'].isnull()
has_product_id = m1 | m2
matched.loc[~has_product_id].groupby('category').count()


In [ ]:
matched.loc[(~matched['category'].isin(['Medications'])) & has_product_id]

In [ ]:
icu_drugs

In [ ]:
def read_ddinter():
    df = pd.read_csv(ddinter_data_dir / "ddinter_downloads_code_B.csv")
    df.Drug_B = df.Drug_B.fillna("")
    df.Drug_B = df.Drug_B.apply(str.lower)
    df.Drug_B = df.Drug_B.fillna("")
    df.Drug_B = df.Drug_B.apply(str.lower)

    df.Drug_A = df.Drug_A.fillna("")
    df.Drug_A = df.Drug_A.apply(str.lower)
    df.Drug_A = df.Drug_A.fillna("")
    df.Drug_A = df.Drug_A.apply(str.lower)
    return df

ddinter = read_ddinter()
cleaned= ddinter
cleaned.loc[(cleaned['Drug_B'].isin( fuzzy_matched['norm_label']) | cleaned['Drug_A'].isin( fuzzy_matched['norm_label'])) & cleaned.Level.isin( ['Major'])]


In [ ]:
def query_for_drug_interactions():
    multimap = cleaned.groupby('Drug_A')['Drug_B'].apply(list).to_dict()
    clauses= []
    for key in multimap.keys():
        first_ids = fuzzy_matched.loc[(fuzzy_matched['norm_label'] == key)]['itemid']

        second_ids = fuzzy_matched.loc[(fuzzy_matched['norm_label'].isin(multimap[key]))]['itemid']
        if first_ids.any() & second_ids.any():
            sql = f"(first_ie.itemid IN ({','.join([str(item) for item in first_ids if item])}) AND second_ie.itemid IN ({','.join([str(item) for item in second_ids if item])}))"
            clauses.append(sql)
    print(len(clauses))
    clause = " OR ".join(clauses)
    print (clause)

    icu = "physionet-data.mimiciv_icu"

    sql = f"""SELECT first_ie.subject_id, first_ie.stay_id, first_ie.itemid, second_ie.itemid, MAX(second_ie.starttime) as dose2_time, count(*)
        FROM `{icu}.inputevents` as first_ie
        INNER JOIN `{icu}.inputevents` as second_ie ON first_ie.stay_id = second_ie.stay_id
        WHERE {clause} AND first_ie.amount > 0
            AND second_ie.amount > 0
            AND first_ie.starttime < second_ie.starttime
            AND DATETIME_DIFF(second_ie.starttime, first_ie.starttime, MINUTE) < 300
        GROUP BY first_ie.subject_id, first_ie.stay_id, first_ie.itemid, second_ie.itemid, second_ie.starttime

    """
    print("\n\n",sql)
    mimic_job = BigQueryClient.auth().query(sql)
    return mimic_job.to_dataframe()

drug_interactions = query_for_drug_interactions()
drug_interactions

In [ ]:
from detectdd.serializer import Serializer

serializer = Serializer()
serializer.write_cohort(drug_interactions)

In [ ]:
serializer.read_cohort()

In [ ]:
drug_interactions